Data Set - https://drive.google.com/file/d/11notSx5jiumfM7r8o6LncIxbD5Q83gYl/view?usp=sharing

The target columns, i.e. the one that needs to be predicted are NFH and NFL. Hyper parameter optimization must also be done

Installing required packages for AutoGluon

In [ ]:
## !pip install -U pip
## !pip install -U setuptools wheel
## !pip install -U "mxnet<2.0.0"
## !pip install autogluon

Import Necessary Libraries

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor
import numpy as np
import pandas as pd

Read the file using the pandas read_csv function

In [ ]:
df = pd.read_csv("csv_nfl_data.csv")

Using the head() function to see first 5 records

In [ ]:
df.head()

,nfl_data,Date(IST),NFO,NFH,NFL,NFC,FIIB,FIIS,FIIN,DIIB,DIIS,DIIN,August,December,CAD,DAD,DOD,NDAD,Currey,Flow,Shine,Vega
0,1,02-01-2020,12115.67324,12177.11996,12151.57155,12216.27237,2670.78,1982.02,688.76,3490.16,3426.21,63.95,1356.0,482.0,-0.01,0.397,-0.98,0.72,-0.0520,0.0399,-0.10,-1.309999
1,2,03-01-2020,12195.40645,12275.48738,12173.35595,12251.19693,4514.35,3251.30,1263.05,2750.87,3780.07,-1029.20,879.0,973.0,-0.23,-0.510,-1.33,0.46,0.4430,0.5648,0.08,1.550000
2,3,06-01-2020,12272.92902,12299.12704,12181.88753,12228.90326,3732.00,3835.84,-103.84,3778.78,3802.48,-23.70,360.0,1502.0,-0.17,0.015,-0.21,-0.07,0.0877,-0.0690,-0.18,-0.170000
3,4,07-01-2020,12237.30935,12242.22714,12030.37114,12051.96001,3911.17,4593.40,-682.23,4205.16,3893.97,311.19,1248.0,578.0,-0.21,-0.050,-0.02,-1.79,-0.0437,-0.0208,0.08,-0.060000
4,5,08-01-2020,12149.13785,12192.47472,11985.55562,12017.34354,4109.04,4624.89,-515.85,5162.02,4413.62,748.40,725.0,1109.0,0.56,0.305,0.54,-0.59,-0.2980,-0.6992,-0.65,-0.340000


We are splitting the dataset into training and testing set

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.30, random_state=0)

First Let us predict for the dependent variable NFL using the AutoGluon Library

In [ ]:
train_data = TabularDataset(train)
subsample_size = 250
train_data = train_data.sample(n=subsample_size, random_state=0)
train_data.head()

,nfl_data,Date(IST),NFO,NFH,NFL,NFC,FIIB,FIIS,FIIN,DIIB,DIIS,DIIN,August,December,CAD,DAD,DOD,NDAD,Currey,Flow,Shine,Vega
72,73,24-04-2020,9238.821198,9342.887445,9183.767428,9341.607717,5668.62,5875.91,-207.29,3903.74,4897.72,-993.98,541.0,1328.0,0.32,0.108,0.56,1.78,0.2800,-0.0773,0.36,-5.450001
184,185,08-10-2020,11712.471060,11800.455530,11684.151000,11786.450030,6586.24,5607.87,978.37,4992.59,4972.74,19.85,828.0,1061.0,0.41,0.205,1.08,1.62,0.0300,0.3817,0.32,-1.699999
75,76,29-04-2020,9395.180215,9439.832542,9325.009629,9400.931427,5733.65,5011.57,722.08,4303.84,4225.17,78.67,1144.0,696.0,3.31,0.768,1.60,0.39,-0.4670,0.3596,-0.41,-2.340000
43,44,06-03-2020,11199.759730,11273.350640,11072.696760,11156.540490,6745.54,10340.38,-3594.84,6861.39,4317.61,2543.78,378.0,1535.0,-1.24,-0.334,-1.13,-3.60,0.4975,-1.2618,0.07,2.320000
130,131,21-07-2020,10936.474760,11021.487210,10900.465860,11005.399710,8209.13,5943.25,2265.88,3465.53,4192.92,-727.39,1030.0,848.0,1.85,0.205,0.74,-0.29,-0.2538,0.3229,0.11,0.380001


In [ ]:
label = 'NFH'
print("Summary of NFH variable: \n", train_data[label].describe())

Summary of NFH variable: 
 count      250.000000
mean     12594.314655
std       2110.332599
min       8367.912828
25%      11243.345045
50%      12145.369420
75%      14727.204542
max      15921.677430
Name: NFH, dtype: float64


In [ ]:
save_path = 'agModels-predictNFH'
predictor = TabularPredictor(label=label, path=save_path).fit(train_data)

Beginning AutoGluon training ...
AutoGluon will save models to "agModels-predictNFH/"
AutoGluon Version:  0.2.0
Train Data Rows:    250
Train Data Columns: 21
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (15921.677430000002, 8367.912828, 12594.31465, 2110.3326)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    12594.64 MB
	Train Data (Original)  Memory Usage: 0.06 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFea

In [ ]:
test_data = TabularDataset(test)
y_test = test_data[label] 
test_data_nolab = test_data.drop(columns=[label])
test_data_nolab.head()

,nfl_data,Date(IST),NFO,NFL,NFC,FIIB,FIIS,FIIN,DIIB,DIIS,DIIN,August,December,CAD,DAD,DOD,NDAD,Currey,Flow,Shine,Vega
6,7,10-01-2020,12093.353330,12035.414370,12117.862140,4679.66,4101.38,578.28,4438.75,4690.49,-251.74,1024.0,811.0,0.02,0.0500,-0.69,-0.51,-0.2446,-0.1071,0.25,0.020000
259,260,02-02-2021,13696.923540,13520.637430,13973.814830,13585.57,7404.01,6181.56,7755.23,9790.43,-2035.20,1235.0,715.0,0.53,0.3257,1.86,4.00,-0.2157,0.2981,-0.20,-4.680000
106,107,16-06-2020,9837.276863,9688.136896,9873.176784,4608.09,6086.61,-1478.52,4742.48,3580.97,1161.51,847.0,1036.0,1.26,0.4890,0.27,1.16,0.2560,0.2257,-0.36,-0.730003
215,216,23-11-2020,12930.114790,12844.122060,12938.833430,12442.96,7704.52,4738.44,3909.28,6853.33,-2944.05,1125.0,793.0,0.25,-0.0410,1.21,-0.32,0.0905,0.1258,-1.04,-1.040001
90,91,22-05-2020,8960.015080,8940.828990,9028.775555,4404.88,5758.78,-1353.90,4164.03,4508.19,-344.16,667.0,1103.0,0.13,0.0590,0.30,1.35,0.4015,-0.2143,0.06,-1.370001


In [ ]:
predictor = TabularPredictor.load(save_path)
y_pred = predictor.predict(test_data)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: root_mean_squared_error on test data: -115.39264689589879
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "root_mean_squared_error": -115.39264689589879,
    "mean_squared_error": -13315.462957641585,
    "mean_absolute_error": -53.21431215451398,
    "r2": 0.9971976923297129,
    "pearsonr": 0.9987423185843748,
    "median_absolute_error": -27.113032343751
}


Predictions:  
 6      12156.879883
259    13904.954102
106    10009.573242
215    13067.146484
90      9142.973633
           ...     
271    15329.813477
113    10534.306641
103    10340.160156
353    15859.124023
248    14743.550781
Name: NFH, Length: 108, dtype: float32


Hyperparameter Tuning

In [ ]:
import autogluon.core as ag

metric = 'rmse'
val_data = test

nn_options = {  # specifies non-default hyperparameter values for neural network models
    'num_epochs': 10,  # number of training epochs (controls training time of NN models)
    'learning_rate': ag.space.Real(1e-4, 1e-2, default=5e-4, log=True),  # learning rate used in training (real-valued hyperparameter searched on log-scale)
    'activation': ag.space.Categorical('relu', 'softrelu', 'tanh'),  # activation function used in NN (categorical hyperparameter, default = first entry)
    'layers': ag.space.Categorical([100], [1000], [200, 100], [300, 200, 100]),  # each choice for categorical hyperparameter 'layers' corresponds to list of sizes for each NN layer to use
    'dropout_prob': ag.space.Real(0.0, 0.5, default=0.1),  # dropout probability (real-valued hyperparameter)
}


gbm_options = {  # specifies non-default hyperparameter values for lightGBM gradient boosted trees
    'num_boost_round': 100,  # number of boosting rounds (controls training time of GBM models)
    'num_leaves': ag.space.Int(lower=26, upper=66, default=36),  # number of leaves in trees (integer hyperparameter)
}

hyperparameters = {  # hyperparameters of each model type
                   'GBM': gbm_options,
                   'NN': nn_options,  # NOTE: comment this line out if you get errors on Mac OSX
                  }  # When these keys are missing from hyperparameters dict, no models of that type are trained

time_limit = 2*60  # train various models for ~2 min
num_trials = 5  # try at most 5 different hyperparameter configurations for each type of model
search_strategy = 'auto'  # to tune hyperparameters using Bayesian optimization routine with a local scheduler

hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy,
}

predictor = TabularPredictor(label=label, eval_metric=metric).fit(
    train_data, tuning_data=val_data, time_limit=time_limit,
    hyperparameters=hyperparameters, hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
)

No path specified. Models will be saved in: "AutogluonModels/ag-20210711_074637/"
Beginning AutoGluon training ... Time limit = 120s
AutoGluon will save models to "AutogluonModels/ag-20210711_074637/"
AutoGluon Version:  0.2.0
Train Data Rows:    250
Train Data Columns: 21
Tuning Data Rows:    108
Tuning Data Columns: 21
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (15921.677430000002, 8367.912828, 12594.31465, 2110.3326)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    12566.1 MB
	Train Data (Original)  Memory Usage: 0.08 MB (0.0% of available memory)
	Inferring dat

Fitted model: LightGBM/T0 ...
	-219.906	 = Validation root_mean_squared_error score
	0.24s	 = Training runtime
	0.0s	 = Validation runtime
Fitted model: LightGBM/T1 ...
	-666.4744	 = Validation root_mean_squared_error score
	0.2s	 = Training runtime
	0.0s	 = Validation runtime
Fitted model: LightGBM/T2 ...
	-300.9967	 = Validation root_mean_squared_error score
	0.24s	 = Training runtime
	0.0s	 = Validation runtime
Fitted model: LightGBM/T3 ...
	-266.4901	 = Validation root_mean_squared_error score
	0.22s	 = Training runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM/T4 ...
	-151.4622	 = Validation root_mean_squared_error score
	0.26s	 = Training runtime
	0.0s	 = Validation runtime
Hyperparameter tuning model: NeuralNetMXNet ...


Fitted model: NeuralNetMXNet/T0 ...
	-1924.012	 = Validation root_mean_squared_error score
	0.35s	 = Training runtime
	0.0s	 = Validation runtime
Fitted model: NeuralNetMXNet/T1 ...
	-4368.5576	 = Validation root_mean_squared_error score
	1.86s	 = Training runtime
	0.01s	 = Validation runtime
Fitted model: NeuralNetMXNet/T2 ...
	-663.6249	 = Validation root_mean_squared_error score
	0.6s	 = Training runtime
	0.01s	 = Validation runtime


Fitted model: NeuralNetMXNet/T3 ...
	-2350.1711	 = Validation root_mean_squared_error score
	1.07s	 = Training runtime
	0.01s	 = Validation runtime
Fitted model: NeuralNetMXNet/T4 ...
	-633.98	 = Validation root_mean_squared_error score
	0.6s	 = Training runtime
	0.01s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ... Training model for up to 119.69s of the 107.06s of remaining time.
	-151.3409	 = Validation root_mean_squared_error score
	0.47s	 = Training runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 13.46s ...
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20210711_074637/")


See the predictions

In [ ]:
y_pred = predictor.predict(test_data_nolab)
print("Predictions:  ", list(y_pred)[:5])
perf = predictor.evaluate(test_data, auxiliary_metrics=False)

Predictions:   [12143.6826171875, 14067.330078125, 10025.71875, 13122.30859375, 8962.9208984375]


Evaluation: root_mean_squared_error on test data: -151.34086254521978
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "root_mean_squared_error": -151.34086254521978
}


In [ ]:
results = predictor.fit_summary()


*** Summary of fit() ***
Estimated performance of each model:
                  model    score_val  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2  -151.340863       0.015739  1.326721                0.000698           0.467247            2       True         11
1           LightGBM/T4  -151.462184       0.003905  0.261623                0.003905           0.261623            1       True          5
2           LightGBM/T0  -219.906043       0.003252  0.240374                0.003252           0.240374            1       True          1
3           LightGBM/T3  -266.490086       0.005049  0.224519                0.005049           0.224519            1       True          4
4           LightGBM/T2  -300.996658       0.003490  0.239571                0.003490           0.239571            1       True          3
5     NeuralNetMXNet/T4  -633.979980       0.006955  0.599045                0.006955           0.

In [ ]:
predictor.get_model_best()

'WeightedEnsemble_L2'

In [ ]:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-151.340863,-151.340863,0.222054,0.015739,1.326721,0.003114,0.000698,0.467247,2,True,11
1,LightGBM/T4,-151.462184,-151.462184,0.007334,0.003905,0.261623,0.007334,0.003905,0.261623,1,True,5
2,LightGBM/T0,-219.906043,-219.906043,0.006421,0.003252,0.240374,0.006421,0.003252,0.240374,1,True,1
3,LightGBM/T3,-266.490086,-266.490086,0.005275,0.005049,0.224519,0.005275,0.005049,0.224519,1,True,4
4,LightGBM/T2,-300.996658,-300.996658,0.008297,0.003490,0.239571,0.008297,0.003490,0.239571,1,True,3
5,NeuralNetMXNet/T4,-633.979967,-633.979980,0.199617,0.006955,0.599045,0.199617,0.006955,0.599045,1,True,10
6,NeuralNetMXNet/T2,-663.624955,-663.624939,0.211607,0.011137,0.597850,0.211607,0.011137,0.597850,1,True,8
7,LightGBM/T1,-666.474364,-666.474364,0.008587,0.003109,0.204492,0.008587,0.003109,0.204492,1,True,2
8,NeuralNetMXNet/T0,-1924.012046,-1924.011963,0.193759,0.004947,0.350993,0.193759,0.004947,0.350993,1,True,6
9,NeuralNetMXNet/T3,-2350.170941,-2350.171143,0.232470,0.014500,1.072001,0.232470,0.014500,1.072001,1,True,9


Next we do predictions for NFL

In [ ]:
label = 'NFL'
print("Summary of NFL variable: \n", train_data[label].describe())

Summary of NFL variable: 
 count      250.000000
mean     12410.702142
std       2135.219146
min       7783.537946
25%      11073.610950
50%      12029.433770
75%      14485.435388
max      15814.822030
Name: NFL, dtype: float64


In [ ]:
save_path = 'agModels-predictNFL'
predictor = TabularPredictor(label=label, path=save_path).fit(train_data)

Beginning AutoGluon training ...
AutoGluon will save models to "agModels-predictNFL/"
AutoGluon Version:  0.2.0
Train Data Rows:    250
Train Data Columns: 21
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (15814.822030000001, 7783.5379459999995, 12410.70214, 2135.21915)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    12550.37 MB
	Train Data (Original)  Memory Usage: 0.06 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting A

[1000]	train_set's rmse: 48.2634	valid_set's rmse: 208.834
[2000]	train_set's rmse: 16.4739	valid_set's rmse: 203.677
[3000]	train_set's rmse: 6.52907	valid_set's rmse: 202.555
[4000]	train_set's rmse: 2.69264	valid_set's rmse: 202.231
[5000]	train_set's rmse: 1.20694	valid_set's rmse: 202.139
[6000]	train_set's rmse: 0.552856	valid_set's rmse: 202.09
[7000]	train_set's rmse: 0.262071	valid_set's rmse: 202.067
[8000]	train_set's rmse: 0.126309	valid_set's rmse: 202.061
[9000]	train_set's rmse: 0.0601756	valid_set's rmse: 202.053
[10000]	train_set's rmse: 0.0289124	valid_set's rmse: 202.05


	-202.05	 = Validation root_mean_squared_error score
	3.21s	 = Training runtime
	0.04s	 = Validation runtime
Fitting model: LightGBM ...
	-106.7204	 = Validation root_mean_squared_error score
	0.38s	 = Training runtime
	0.0s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-67.4423	 = Validation root_mean_squared_error score
	0.85s	 = Training runtime
	0.1s	 = Validation runtime
Fitting model: CatBoost ...
	-195.3425	 = Validation root_mean_squared_error score
	7.54s	 = Training runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-63.0571	 = Validation root_mean_squared_error score
	0.75s	 = Training runtime
	0.1s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	-12352.8309	 = Validation root_mean_squared_error score
	2.8s	 = Training runtime
	0.03s	 = Validation runtime
Fitting model: XGBoost ...
	-103.9773	 = Validation root_mean_squared_error score
	0.39s	 = Training runtime
	0.0s	 = Validation runtime
Fitting model: NeuralNetMXNet ...
	-225.5445	

In [ ]:
test_data = TabularDataset(test)
y_test = test_data[label] 
test_data_nolab = test_data.drop(columns=[label])
test_data_nolab.head()

,nfl_data,Date(IST),NFO,NFH,NFC,FIIB,FIIS,FIIN,DIIB,DIIS,DIIN,August,December,CAD,DAD,DOD,NDAD,Currey,Flow,Shine,Vega
6,7,10-01-2020,12093.353330,12173.394190,12117.862140,4679.66,4101.38,578.28,4438.75,4690.49,-251.74,1024.0,811.0,0.02,0.0500,-0.69,-0.51,-0.2446,-0.1071,0.25,0.020000
259,260,02-02-2021,13696.923540,14115.671350,13973.814830,13585.57,7404.01,6181.56,7755.23,9790.43,-2035.20,1235.0,715.0,0.53,0.3257,1.86,4.00,-0.2157,0.2981,-0.20,-4.680000
106,107,16-06-2020,9837.276863,9983.324733,9873.176784,4608.09,6086.61,-1478.52,4742.48,3580.97,1161.51,847.0,1036.0,1.26,0.4890,0.27,1.16,0.2560,0.2257,-0.36,-0.730003
215,216,23-11-2020,12930.114790,13021.950080,12938.833430,12442.96,7704.52,4738.44,3909.28,6853.33,-2944.05,1125.0,793.0,0.25,-0.0410,1.21,-0.32,0.0905,0.1258,-1.04,-1.040001
90,91,22-05-2020,8960.015080,9093.216220,9028.775555,4404.88,5758.78,-1353.90,4164.03,4508.19,-344.16,667.0,1103.0,0.13,0.0590,0.30,1.35,0.4015,-0.2143,0.06,-1.370001


In [ ]:
predictor = TabularPredictor.load(save_path)
y_pred = predictor.predict(test_data)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: root_mean_squared_error on test data: -160.9041998020301
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "root_mean_squared_error": -160.9041998020301,
    "mean_squared_error": -25890.161513931613,
    "mean_absolute_error": -79.21479401996524,
    "r2": 0.9947823768855294,
    "pearsonr": 0.9976717503087044,
    "median_absolute_error": -34.1432345000012
}


Predictions:  
 6      12057.388672
259    13864.142578
106     9714.517578
215    12910.765625
90      8921.041016
           ...     
271    15178.943359
113    10350.906250
103    10106.503906
353    15738.392578
248    14522.099609
Name: NFL, Length: 108, dtype: float32


In [ ]:
import autogluon.core as ag

metric = 'rmse'
val_data = test

nn_options = {  # specifies non-default hyperparameter values for neural network models
    'num_epochs': 10,  # number of training epochs (controls training time of NN models)
    'learning_rate': ag.space.Real(1e-4, 1e-2, default=5e-4, log=True),  # learning rate used in training (real-valued hyperparameter searched on log-scale)
    'activation': ag.space.Categorical('relu', 'softrelu', 'tanh'),  # activation function used in NN (categorical hyperparameter, default = first entry)
    'layers': ag.space.Categorical([100], [1000], [200, 100], [300, 200, 100]),  # each choice for categorical hyperparameter 'layers' corresponds to list of sizes for each NN layer to use
    'dropout_prob': ag.space.Real(0.0, 0.5, default=0.1),  # dropout probability (real-valued hyperparameter)
}


gbm_options = {  # specifies non-default hyperparameter values for lightGBM gradient boosted trees
    'num_boost_round': 100,  # number of boosting rounds (controls training time of GBM models)
    'num_leaves': ag.space.Int(lower=26, upper=66, default=36),  # number of leaves in trees (integer hyperparameter)
}

hyperparameters = {  # hyperparameters of each model type
                   'GBM': gbm_options,
                   'NN': nn_options,  # NOTE: comment this line out if you get errors on Mac OSX
                  }  # When these keys are missing from hyperparameters dict, no models of that type are trained

time_limit = 2*60  # train various models for ~2 min
num_trials = 5  # try at most 5 different hyperparameter configurations for each type of model
search_strategy = 'auto'  # to tune hyperparameters using Bayesian optimization routine with a local scheduler

hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy,
}

predictor = TabularPredictor(label=label, eval_metric=metric).fit(
    train_data, tuning_data=val_data, time_limit=time_limit,
    hyperparameters=hyperparameters, hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
)

No path specified. Models will be saved in: "AutogluonModels/ag-20210711_074722/"
Beginning AutoGluon training ... Time limit = 120s
AutoGluon will save models to "AutogluonModels/ag-20210711_074722/"
AutoGluon Version:  0.2.0
Train Data Rows:    250
Train Data Columns: 21
Tuning Data Rows:    108
Tuning Data Columns: 21
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (15814.822030000001, 7783.5379459999995, 12410.70214, 2135.21915)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    12555.64 MB
	Train Data (Original)  Memory Usage: 0.08 MB (0.0% of available memory)
	Infe

Fitted model: LightGBM/T0 ...
	-256.2419	 = Validation root_mean_squared_error score
	0.24s	 = Training runtime
	0.0s	 = Validation runtime
Fitted model: LightGBM/T1 ...
	-719.0885	 = Validation root_mean_squared_error score
	0.2s	 = Training runtime
	0.0s	 = Validation runtime
Fitted model: LightGBM/T2 ...
	-345.0307	 = Validation root_mean_squared_error score
	0.22s	 = Training runtime
	0.0s	 = Validation runtime
Fitted model: LightGBM/T3 ...
	-317.6946	 = Validation root_mean_squared_error score
	0.23s	 = Training runtime
	0.0s	 = Validation runtime
Fitted model: LightGBM/T4 ...
	-177.5943	 = Validation root_mean_squared_error score
	0.29s	 = Training runtime
	0.0s	 = Validation runtime
Hyperparameter tuning model: NeuralNetMXNet ...


Fitted model: NeuralNetMXNet/T0 ...
	-1818.9602	 = Validation root_mean_squared_error score
	0.35s	 = Training runtime
	0.01s	 = Validation runtime
Fitted model: NeuralNetMXNet/T1 ...
	-4481.7866	 = Validation root_mean_squared_error score
	1.9s	 = Training runtime
	0.01s	 = Validation runtime
Fitted model: NeuralNetMXNet/T2 ...
	-858.9706	 = Validation root_mean_squared_error score
	1.09s	 = Training runtime
	0.01s	 = Validation runtime


Fitted model: NeuralNetMXNet/T3 ...
	-924.6284	 = Validation root_mean_squared_error score
	1.22s	 = Training runtime
	0.01s	 = Validation runtime
Fitted model: NeuralNetMXNet/T4 ...
	-2385.0588	 = Validation root_mean_squared_error score
	0.62s	 = Training runtime
	0.01s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ... Training model for up to 119.79s of the 106.83s of remaining time.
	-175.6037	 = Validation root_mean_squared_error score
	0.47s	 = Training runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 13.67s ...
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20210711_074722/")


In [ ]:
y_pred = predictor.predict(test_data_nolab)
print("Predictions:  ", list(y_pred)[:5])
perf = predictor.evaluate(test_data, auxiliary_metrics=False)

Predictions:   [12065.0400390625, 13764.560546875, 9714.82421875, 12940.2626953125, 8802.130859375]


Evaluation: root_mean_squared_error on test data: -175.60367689087823
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "root_mean_squared_error": -175.60367689087823
}


In [ ]:
results = predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                  model    score_val  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2  -175.603677       0.016587  1.980473                0.000634           0.473131            2       True         11
1           LightGBM/T4  -177.594264       0.003641  0.288986                0.003641           0.288986            1       True          5
2           LightGBM/T0  -256.241934       0.003635  0.236598                0.003635           0.236598            1       True          1
3           LightGBM/T3  -317.694607       0.003406  0.226937                0.003406           0.226937            1       True          4
4           LightGBM/T2  -345.030740       0.003211  0.217197                0.003211           0.217197            1       True          3
5           LightGBM/T1  -719.088529       0.003373  0.198557                0.003373           0.

In [ ]:
predictor.get_model_best()

'WeightedEnsemble_L2'

In [ ]:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-175.603677,-175.603677,0.252888,0.016587,1.980473,0.003239,0.000634,0.473131,2,True,11
1,LightGBM/T4,-177.594264,-177.594264,0.018520,0.003641,0.288986,0.018520,0.003641,0.288986,1,True,5
2,LightGBM/T0,-256.241934,-256.241934,0.006078,0.003635,0.236598,0.006078,0.003635,0.236598,1,True,1
3,LightGBM/T3,-317.694607,-317.694607,0.005266,0.003406,0.226937,0.005266,0.003406,0.226937,1,True,4
4,LightGBM/T2,-345.030740,-345.030740,0.005686,0.003211,0.217197,0.005686,0.003211,0.217197,1,True,3
5,LightGBM/T1,-719.088529,-719.088529,0.005409,0.003373,0.198557,0.005409,0.003373,0.198557,1,True,2
6,NeuralNetMXNet/T2,-858.970637,-858.970642,0.230865,0.012323,1.088309,0.230865,0.012323,1.088309,1,True,8
7,NeuralNetMXNet/T3,-924.628330,-924.628357,0.231129,0.012312,1.218356,0.231129,0.012312,1.218356,1,True,9
8,NeuralNetMXNet/T0,-1818.960233,-1818.960205,0.192407,0.005577,0.350535,0.192407,0.005577,0.350535,1,True,6
9,NeuralNetMXNet/T4,-2385.058965,-2385.058838,0.198873,0.012564,0.617691,0.198873,0.012564,0.617691,1,True,10
